In [1]:
import pandas as pd
import psycopg2
import user_credentials

In [2]:
# Get user input
country = 'France'
grade_lower = 'V5'
grade_upper = 'V7'
style = 'Neutral'

In [3]:
# Choose desired grading scale
try:
    if '.' in grade_lower:
        selected_scale = 'grade_yds'
    elif 'v' in grade_lower.lower():
        selected_scale = 'grade_v'
    else:
        selected_scale = 'grade_fra'
except TypeError:
    pass    # Handle bad grade error

In [4]:
# Connect to the climbing_db database
database_name = "climbing_db"
try:
    conn = psycopg2.connect(database = database_name, user = user_credentials.username, password = user_credentials.password, 
                            host = "localhost", port = "5432")
    cursor = conn.cursor()
except psycopg2.errors.OperationalError:
    print("Database connection not successful")

In [5]:
# Query routes data
cursor.execute(f'''
SELECT c.country, r.crag, r.sector, r.name, r.grade_mean, r.rating_tot, r.style, g.{selected_scale}
FROM countries AS c
JOIN routes AS r ON c.country_id = r.country
JOIN grades AS g on r.grade_mean = g.grade_id
WHERE c.country = '{country}'
''')
data = cursor.fetchall()

# Extract the data into a dataframe
df = pd.DataFrame(data, columns=['country', 'crag', 'sector', 'route name', 'grade_mean', 'rating', 'style', 'grade'])

In [6]:
cursor.close()
conn.close()

In [ ]:
# Filter for style
df = df[df['style'] == style]
df

In [ ]:
# Filter for grade range
min_grade = grades_df[grades_df[selected_scale] == grade_lower].index.min()
max_grade = grades_df[grades_df[selected_scale] == grade_upper].index.max()
df = df[(df['grade'] >= min_grade) & (df['grade'] <= max_grade)]
df

In [ ]:
# Keep crags with at least X routes
df.groupby('crag').count() > 5